In [30]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
a,b=6,9
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((a * b, 3), np.float32)
objp[:, :2] = np.mgrid[0:a, 0:b].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('img/images/*.jpg')
#images = glob.glob('img/chessboard/patron_1.jpg')
for fname in images:
    img = cv.imread(fname)
    #print(img.shape)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(img, (a,b), None)
    print(ret)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (a,b), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
cv.drawChessboardCorners(img, (a,b), corners2, ret)
cv.imshow('img '+str(fname), cv.resize(img,(600,400)))
cv.waitKey(100)
cv.destroyAllWindows()

True
True


In [37]:

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(mtx)

[[808.31048036   0.         403.1623161 ]
 [  0.         809.77232795 222.32429755]
 [  0.           0.           1.        ]]


In [32]:
img = cv.imread('img/images/far/image_0.jpg')
#img = cv.imread('img/image_5.jpg')
h,  w = img.shape[:2]
print(h,w)

480 752


In [33]:

newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
print(roi)

(41, 45, 646, 360)


In [34]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
#dst = dst[y:y+h, x:x+w]
cv.imshow('img '+str(fname), cv.resize(dst,(600,400)))
cv.waitKey(1000)
cv.destroyAllWindows()
cv.imwrite('img/calibresult_1.png', dst)

True

In [35]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_CUBIC)
# crop the image
#x, y, w, h = roi
roi=roi
dst = dst[y:y+h, x:x+w]
cv.imshow('img '+str(fname), cv.resize(dst,(600,400)))
cv.waitKey(1000)
cv.destroyAllWindows()
cv.imwrite('img/calibresult_2.png', dst)

True

In [36]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.038654827558090514
